In [1]:
import os
import pickle

import pandas as pd

In [ ]:
texts = {}
trees = {}
results = {}
names = os.listdir('../русский авторазметчик эде')
for name in names:
    with open('../русский авторазметчик эде' + '/' + name, 'br') as f:
            obj = pickle.load(f)
            if 'tree' in name:
                trees[name] = obj
            else:
                if 'result' in name:
                    results[name] = obj
                else:
                    texts[name] = obj

In [ ]:
def match_lemmas(tokens, lemmas):
    straight_lemmas = []
    for sentence in lemmas:
        for lemma in sentence:
            straight_lemmas.append(lemma)
    tokens_texts = []
    for token in tokens:
        tokens_texts.append(token.text)
    return dict(zip(tokens_texts, straight_lemmas))

In [ ]:
def lemmatize(tree, tokens, lemmas):
    words = []
    for token in tokens:
        if tree.start <= token.begin and token.end <= tree.end:
            words.append(lemmas[token.text])
    return words

In [ ]:
def rule_1(tree, tokens, lemmas):
    return tree.relation == 'attribution'


def rule_2(tree, tokens, lemmas):
    if tree.relation == 'elaboration':
        lemmas = lemmatize(tree, tokens, lemmas)
        if 'который' in lemmas:
            return True
    return False


def rule_3(tree, tokens, lemmas):
    if tree.relation != 'elementary':
        lemmas = lemmatize(tree, tokens, lemmas)
        if len(c_verbs & set(lemmas)) != 0:
            return True
        else:
            return False
    else:
        return False

rules = [rule_1, rule_2, rule_3]

In [ ]:
def conditions_failed(tree, rules, tokens, lemmas):
    for rule in rules:
        if rule(tree, tokens, lemmas):
            return True
    return False

In [ ]:
def corrected(tree, rules, tokens, lemmas):
    if conditions_failed(tree, rules, tokens, lemmas):
        return delete_relation(tree)
    else:
        return tree

In [ ]:
def delete_relation(tree):
    if tree.nuclearity == 'SN':
        attr = tree.left
        attr.relation = 'elementary'
        n = left_n_leaf(tree.right)
        if (n.start - attr.end) > 1:
            n_copy = copy.deepcopy(n)
            n.right = n_copy 
            n.left = attr
            n.relation = 'same-unit'
        else:
            n.start = attr.start
            n.text = attr.text + n.text
        return tree.right
            
    else:
        attr = tree.right
        attr.relation = 'elementary'
        n = right_n_leaf(tree.left)
        if (attr.start - n.end) > 1:
            n_copy = copy.deepcopy(n)
            n.left = n_copy
            n.right = attr
            n.relation = 'same-unit'
        else:           
            n.end = attr.end
            n.text += attr.text
        return tree.left

In [ ]:
def left_n_leaf(tree):
    if tree.left:
        if tree.nuclearity == 'SN':
            return left_n_leaf(tree.right)
        else:
            return left_n_leaf(tree.left)
    else:
        return tree

In [ ]:
def right_n_leaf(tree):
    if tree.right:
        if tree.nuclearity == 'NS':
            return right_n_leaf(tree.left)
        else:
            return right_n_leaf(tree.right)
    else:
        return tree

In [ ]:
def segmentation(tree, text, segments, rules, tokens, lemmas):
    if tree.relation != 'elementary':
        tree.left = corrected(tree.left, rules, tokens, lemmas)
        tree.right = corrected(tree.right, rules, tokens, lemmas)
        segmentation(tree.left, text, segments, rules, tokens, lemmas)
        segmentation(tree.right, text, segments, rules, tokens, lemmas)
    else:
        segments.append(text[tree.start:tree.end])

In [ ]:
def texts_segmentation(rules):
    segments =  []
    for key in results:
    #for i in range(2):
        #key = list(results.keys())[i]
        tree = results[key]['rst'][0]
        text = results[key]['text']
        tokens = results[key]['tokens']
        lemmas = results[key]['lemma']
        lemmas = match_lemmas(tokens, lemmas)
        segmentation(tree, text, segments, rules, tokens, lemmas)
        # для текстов b021,b023 и b050 автосегментатотор почему-то вернул два дерева вместо одного, поэтому нужно приделать второе дерево(там одно эде)
        #очень костыльно конечно, но думаю так не должно быть в других текстах, какой-то сбой
        if len(results[key]['rst']) > 1:
            segments.append(results[key]['rst'][1].text)
        segments.append('\n')
    return segments

In [ ]:
new_array=range(len(rules))
power_set=[[]]
for x in new_array:
    for i in range(len(power_set)):
        tmp_list = power_set[i].copy()
        tmp_list.append(x)
        power_set.append(tmp_list)
power_set = power_set[1:]

In [ ]:
for s in power_set:
    current_rules = []
    for i in s:
        current_rules.append(rules[i])
    print(current_rules)
    filename_raw = str(current_rules).split()
    filename = []
    for word in filename_raw:
        if 'rule' in word:
            filename.append(word)
    filename = '__'.join(filename)
    filename += '.xlsx'
    segments = texts_segmentation(current_rules)
    print(filename + ' сегментирован')
    exls = pd.DataFrame(segments)
    exls.to_excel(filename, index = False)
    print(filename + ' записан')